In this notebook, we simulate the field created by the TRIF electromagnet and save it as a data set. This data set is going to be used to test our calibration approach.

We model the electromagnet as three magnetic dipoles oriented in the same direction. The dipoles are located in the same plain at the same distance from the axis of symmetry, and their position vectors are at $120^0$ with respect to each other. We also assume that there can be a uniform external magnetic field.

***

## Setup

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
# Functions for the field calculations

def mag(x):
    """Calculates magnitude of vector x"""
    return np.sqrt(np.dot(x,x))

def dipole_field(r, r0, m, a):
    """
    Calculates magnetic field of a single dipole
    r - location of the field
    r0 - position of the dipole
    m - magnitude of the dipole. Negative values correspond to the opposite orientation of  the dipole axis
    a - dipole axis"""
    M = m*a   # Magnetic moment vector
    R = r-r0
    if np.dot(R,R) == 0: #This one is to avoid singularities
        return np.array([0, 0, 0])
    return mu0*(3*R*(np.dot(M,R))/(mag(R)**5) - M/(mag(R)**3))

def total_field(r, m_array: np.array, B0: np.array, dipole_axis: np.array) -> np.array:
    """Calculates the total field at point r
    To simplify the data set generation later, the magnitudes of the three dipoles are placed in array m.
    B0 is the external field.
    dipole_axis is the dipole orientation (same for all three)"""
    B1 = dipole_field(r, R1, m_array[0], dipole_axis)
    B2 = dipole_field(r, R2, m_array[1], dipole_axis)
    B3 = dipole_field(r, R3, m_array[2], dipole_axis)
    return B1+B2+B3+B0

In [5]:
# Constants and the setup parameters

mu0 = 10**-7# magnetic constant
m = 1e8 # Scaling factor magnetic dipole magnitude

# Magnetic dipole position vectors
R1 = np.array([1, 0, 1])
R2 = np.array([-np.cos(np.pi/3), np.sin(np.pi/3), 1])
R3 = np.array([-np.cos(np.pi/3), -np.sin(np.pi/3), 1])

# Magnetic dipole orientation (same for all three)
A = np.array([0, 0, 1]) # It is a global variable

# External magnetic field
B_uniform = np.array([0.,0.,0.])

# The magnetometer location. Magnetic field is calculated here
r0 = np.array([0., 0., 0.])

## Generate data set of magnetic dipole magnitudes.
 In the experiment each dipole magnitude corresponds to a voltage applied to the corresponding coil inside the magnet.

In [7]:
# Apply voltage to a single coil in a linear fashion

N1 = 501 # number of training samples for each dipole
M1 = np.concatenate((np.linspace(-10, 10, N1), np.zeros((N1)), np.zeros(N1)))
M2 = np.concatenate((np.zeros((N1)), np.linspace(-10, 10, N1),  np.zeros(N1)))
M3 = np.concatenate((np.zeros((N1)), np.zeros(N1), np.linspace(-10, 10, N1)))

targets=np.stack((M1, M2, M3), axis=1)

In [11]:
targets[0]

array([-10.,   0.,   0.])

In [12]:
# Aplly random
N2 = 1000 #

M = np.random.uniform(-10, 10, (N2, 3))

targets_train = np.concatenate((targets, M))

In [13]:
targets_train[0]

array([-10.,   0.,   0.])

In [ ]:
B = []

for M in targets_train:
    B.append(total_field(r0, M*m, B_uniform, A))
    
B_train = np.array(B)

Training_set = pd.DataFrame(np.concatenate((B_train, targets_train), axis = 1), columns = ["B_x", "B_y", "B_z", "M_1", "M_2", "M_3"])

# np.save("data\B_train.npy", B_train)
# np.save("data\M_train.npy", targets_train)

In [ ]:
N_test = 100

targets_test = np.random.uniform(-10, 10, (N_test, 3))
B = []

for M in targets_test:
    B.append(total_field(r0, M*m))
    
B_test = np.array(B)

# np.save("data\B_test.npy", B_test)
# np.save("data\M_test.npy", targets_test)
